In [1]:
#크롬드라이버,selenium 설치
#!apt update
#!apt install chromium-chromedriver
!pip install selenium

# 옵션설정
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [2]:
driver = webdriver.Chrome("chromedriver.exe") 

In [17]:
#singer = ['서태지와 아이들','더 블루','잼','듀스','노이즈','룰라','쿨','터보','알이에프','에이치오티',
#         '언타이틀','하모하모','태사자','엔알지','티앤비','젝스키스','베이비복스']
singer=  ['에스이에스','원타임','신화','이글파이브','써클','핑클','코요태','플라이투더스카이','에이포',
          '클릭비','god','문차일드','클레오','티티마']
singer_id =[]

In [18]:
for i in singer:
    driver.get("https://www.melon.com/index.htm")   
    search = driver.find_element_by_xpath('//*[@id="top_search"]') #검색창 xpath
    search.clear() #검색창 clear
    search.send_keys(i+'\n')  #검색창에 가수 입력후 엔터
    
    driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div/div[1]/div/a/strong').click() #검색한 아티스트 클릭
    url = driver.current_url
    id = url.replace('https://www.melon.com/artist/timeline.htm?artistId=', '') #url에 있는 가수 고유번호 저장
    
    singer_id.append(id)
    
    

In [8]:
print(singer_id)

[]


In [19]:
title_ary=[]
album_ary=[]
date_ary=[]
genre_ary=[]
lyrics_ary=[]
lyricist_ary=[]
composer_ary=[]
singer_ary=[]

In [20]:
def search(id,n):   # 가수id를 받아 (제목,가수,가사,앨범,발매일,장르,작사,작곡)  n개 만큼탐색  
    #id에 맞는 가수의 곡 페이지(인기순)으로 이동
    driver.get('https://www.melon.com/artist/song.htm?artistId='+str(id)+'#params%5BlistType%5D=A&params%5BorderBy%5D=POPULAR_SONG_LIST&params%5BartistId%5D='+str(id)+'&po=pageObj&startIndex=1')
    driver.implicitly_wait(1)
    for i in range(1,n+1):

        driver.execute_script('window.scrollTo(0,200)')
        
        # i번째 곡정보 클릭
        driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+ str(i)+']/td[3]/div/div/a[1]').click()
        
        #제목 가수 가사 앨범 발매일 장르 작사 작곡 정보 리스트에 저장
        title=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]')
        title_ary.append(title.text)
    
        singer=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]')
        singer_ary.append(singer.text)
    
        album=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a')
        album_ary.append(album.text)
    
        date=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]')
        date_ary.append(date.text)
    
        genre=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]')
        genre_ary.append(genre.text)
        
        # 가사,작사,작곡이 없는경우가 있어 예외처리
        try:
            lyricist=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[1]/div[2]/div[1]/a')
            lyricist_ary.append(lyricist.text)
        except:
            lyricist_ary.append('NULL')
        
        try:
            composer=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[2]/div[2]/div[1]/a')
            composer_ary.append(composer.text)
        except:
            composer_ary.append('NULL')
        
        try:
            lyrics=driver.find_element_by_xpath('//*[@id="d_video_summary"]')
            lyrics_ary.append(lyrics.text.replace('\n',' '))
        except:
            lyrics_ary.append('NULL')
            
        driver.back() #뒤로가기 ->다시 곡 목록으로 이동
    
    
    
    
    
    
    
    
    

In [21]:
for i in singer_id:
   search(i,10)



In [22]:
import pandas as pd
df=pd.DataFrame({"년도":date_ary,"가수":singer_ary,"제목":title_ary,"앨범":album_ary,"장르":genre_ary,
             "작사가":composer_ary,"작곡가":lyricist_ary,"가사":lyrics_ary})

In [23]:
print(df)

             년도      가수                      제목                       앨범  장르  \
0       1998.11  S.E.S.                  너를 사랑해                 S.E.S. 2  댄스   
1    2001.07.11  S.E.S.   꿈을 모아서 (Just In Love)                 Surprise  댄스   
2       1998.11  S.E.S.        Dreams Come True                 S.E.S. 2  댄스   
3    1997.11.01  S.E.S.  (`Cause) I`m Your Girl                   S.E.S.  댄스   
4    2002.02.15  S.E.S.                     달리기         Choose My Life-U  댄스   
..          ...     ...                     ...                      ...  ..   
135  1999.05.01     티티마             Smile Again               In The Sea  댄스   
136  1999.05.01     티티마                   Loner               In The Sea  댄스   
137  2000.06.15     티티마                 Delight            Iwannabe.....  댄스   
138  1999.05.01     티티마              Come To Me               In The Sea  댄스   
139  2007.12.13     티티마                   메리 메리  ComeBack With Christmas  댄스   

               작사가                작곡가  

In [24]:
df.to_csv('k-pop_2.tsv',index=False,header=None,sep='\t') #tsv파일로 저장